# Microbial Exploration of Spirea Creek, Yellowstone National Park
### Dylan Gagler, Arizona State University

## Introduction

Yellowstone National Park (YNP) is a place of great geochemical and microbiological interest. Due to its geological setting, the park hosts an extraordinary collection of terrestrial hot springs, each of which is home to diverse microbial communities.

These geochemical hot springs typically range between 50-90&deg;C and are full of of typically disagreeable chemical compounds such as hydrogen sulfide (H<sub>2</sub>S) and ammonia (NH<sub>4</sub>), but are they are populated by heat-loving, or **thermophilic**, microbes. These conditions seem inhospitable to humans, but for the single-celled bacteria and archaea in these hot springs, they are quite pleasant. Characterizing the microbes which inhabit these hot springs is important not only for expanding our view of the diversity of life, but also for understanding the genes and proteins that allow life to survive in such extreme environments, which can have downstream impacts on the fields of biotechnology, structural biology, and synthetic biology.

                                                    St. Blucia, YNP
<img src="st_blucia.png" width="300"/>

To further expand our understanding of the geochemistry and microbiology of extreme environments, I studied a set of previously uncharacterized hot springs in the Spirea Creek area of Yellowstone National Park. Four distinct hot springs were sampled and named Heart of Joy, Hat Stew, Green Machine, and Bubble Boy. At first glance, Hat Stew stood out considerably, being the only spring which was lacking obvious photosynthetic pigments. Instead, the inner surface was a cream color, fringed by orange. Hat Stew was measured at 47&deg;C and a pH of 7.9, but displayed no obvious signs of photosynthesis: a strange observation for this temperature and pH regime. 

In addition to geochemical samples, biological samples were taken from the Hat Stew and 3 surrounding springs (Heart of Joy, Green Machine, and Bubble Boy) and frozen for storage. Later, at Arizona State University, DNA was extracted and amplified using PCR and subsequently analyzed using the 16S Illumina Amplicon Protocol with the 515F-806R primer pair.

                                                    Hat Stew, YNP
<img src="hat_stew.png" width="400"/>



                    Heart of Joy (left), Bubble Boy (center), and Green Machine (right)

<img align="left" width="300" height="200" src="HoJ.png">
<img align="right" width="300" height="100" src="GM.png">

<p align="center">
  <img width="300" height="28" src="BB.png">
</p>



# Sequence processing and analysis using QIIME2

**QIIME 2** is a microbiome analysis package which takes raw sequence data as input and outputs. QIIME 2 was used to explore the Spirea Creek biological data.

### Importing data

QIIME 2 uses QIIME artifact files (.qza and .qzv) for all of it's commands, so firstly, the sequence data needs to be imported into QIIME2 and converted into QIIME artifacts. This is done as follows.

In [ ]:
qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path '/Users/dgagler/School/SecondaryProject/Sequencing_Demo/spireacreek_M' \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path demux-paired-end_Spirea.qza

### Demultiplexing
At this point in the analysis, the sequence data is typically demultiplexed, whereby "barcode" tags (genetic identifiers attached to sequences before all the material was sequenced) are used to associate the mixed-up sequences to their parent samples. In the case of the Spirea Creek samples, the sequences were demultiplexed by the sequencing center, so this step was skipped.

### Quality control using fastQC

FastQC is a program for analyzing the quality of genetic sequences. Quality scores are important for downstream trimming of sequence data, which will become apparent shortly. Spirea Creek sequence data were assessed as such.

                                        FastQC quality scores for Heart of Joy
<img src="112_qualityscores.png" width="500"/>

## Denoising via DADA2

Denoising the data involves trimming segments of the sequences which are of lesser quality. To do this, one must assess the quality of the sequences using a tool like FastQC. Considering variations in the sequence quality of the samples, a quality cut-off of 25/26 was chosen and a subsequent a denoising length of 220 bases was selected.

In [ ]:
qiime dada2 denoise-single \
  --i-demultiplexed-seqs demux-paired-end_Spirea.qza \
  --p-trim-left 0 \
  --p-trunc-len 220 \
  --o-representative-sequences rep-seqs-Spirea.qza \
  --o-table table-Spirea.qza \
  --o-denoising-stats stats-Spirea.qza

This code outputs several files, including a representative sequence file, a feature table, and a stats table.

In [ ]:
qiime metadata tabulate \
  --m-input-file stats-spirea.qza \
  --o-visualization stats-spirea.qzv

Which are subsequently used to tabulate a statistics table which can subsequently be visualized by the following command.

In [ ]:
qiime tools view stats-spirea.qzv

## Feature table and feature data summaries
After filtering the data for quality and generating feature tables and statistics summaries, the data is ready to be visualized. 

- The feature-table summarize command will give you information on how many sequences are associated with each sample and with each feature, histograms of those distributions, and some related summary statistics. 

- The feature-table tabulate-seqs command will provide a mapping of feature IDs to sequences.

Running the following commands creates visualization files (.qzv).

In [ ]:
qiime feature-table summarize \
  --i-table table-Spirea.qza \
  --o-visualization table-Spirea.qzv \
  --m-sample-metadata-file spirea-metadata.txt
qiime feature-table tabulate-seqs \
  --i-data rep-seqs-Spirea.qza \
  --o-visualization rep-seqs-Spirea.qzv

In [ ]:
qiime tools view table-Spirea.qzv

<img src="qiimedemo_featuresummary.png" width="650"/>

<img src="qiimedemo_id_featurecount.png" width="650"/>

In [ ]:
qiime tools view rep-seqs-Spirea.qzv

<img src="qiimedemo_repseqs.png" width="1000"/>

## Taxonomic analysis
Taxonomy is assigned to the sequences using a pre-trained Naive Bayes classifier and a feature-classifier plugin. The classifier used was trained on the Greengenes 13_8 99% OTUs, where the sequences have been trimmed to only include 250 bases from the region of the 16S that was sequenced in this analysis (the V4 region, bound by the 515F/806R primer pair). Using this classifier, visualizations of the resulting mapping from sequence to taxonomy are generated.

In [ ]:
qiime feature-classifier classify-sklearn \
  --i-classifier gg-13-8-99-515-806-nb-classifier.qza \
  --i-reads rep-seqs-Spirea.qza \
  --o-classification taxonomy-Spirea.qza

qiime metadata tabulate \
  --m-input-file taxonomy-Spirea.qza \
  --o-visualization taxonomy-Spirea.qzv

In [ ]:
qiime taxa barplot \
  --i-table table-Spirea.qza \
  --i-taxonomy taxonomy-Spirea.qza \
  --m-metadata-file spirea-metadata.txt \
  --o-visualization taxa-bar-plots-Spirea.qzv

qiime tools view taxa-bar-plots-Spirea.qzv

### QIIME 2 taxa bar plots

<img align="left" width="200" height="120" src="level_1.png">
<img align="right" width="300" height="300" src="level_3.png">

<p align="center">
  <img width="250" height="200" src="level_2.png">
</p>


As you can see, the resulting bar plots are visually messy and difficult to interpret. Downloading the associated taxa data as .csv files allows personalized visualizations to be created. The following plots were generated using the matplotlib and seaborn modules in Python.

## Personal visualizations

<img align="left" width="450" height="300" src="spirea_phylum.png">

<img align="right" width="450" height="300" src="spirea_genera.png">



## Interpreting results

At a glance, the microbial composition of these 4 hot springs is diverse. These plots show microbial composition of the four hot springs at Spirea Creek at two taxonomical levels: phylum (general) and genus (specific). The 'Other' category contains taxa with less than 4% representation in a given hot spring. 

#### Phylum distributions

At the phylum level, Hat Stew is dominated by Proteobacteria but also contains a non-trivial population of Firmicutes and a small community of Actinobacteria. The other 3 springs have far much smaller populations of Chloroflexi and only 1 other spring, Green Machine, contains Firmicutes. Furthermore. these other 3 springs have substantial populations of Chloroflexi, which Hat Stew does not. Cyanobacteria and Planctomycetes make an appearance in only 2 of the hot springs, Heart of Joy and Bubble Boy. Chlorobi are unique to Heart of Joy, Actinobacteria are unique to Hat Stew, Armatimonadetes are unique to Green Machine, and Eurarchaeota are unique to Bubble Boy. 

#### Genus distributions

At the genus level, organismal resolution becomes rather low for these springs (evidenced by the large size of the 'Other' category) but a few details still stand out. Most notably, Hat Stew is totally dominated by Phyllobacterium, which explains the large fraction of Proteobacteria seen at the phylum level. Phyllobacterium are not a well studied group, but they seem to be aerobic and associated with nitrogen-fixation and root nodules. Also interestingly, they are commonly white or cream colored, as Hat Stew very obviously was. Hat Stew also contains a population of Methylobacterium, which use methanol, formate, formaldehyde, and other 1-carbon compounds for both energy and biomass. They are notably associated with plant decomposition. The vague associations with both root nodules and plant decomposition suggest that the reason for Hat Stew's strange microbial composition might be due to an influx of plant matter into the spring, which provide the sustence for certain ways of metabolic life in the hot spring. The other 3 springs all contain notable populations of Roseiflexus, a sub-classification of Chloroflexi. Roseiflexus are thermophilic, photosynthetic bacteria. This matches our initial observation of photosynthetic pigments in these springs.

In conclusion, Hat Stew is a microbiologically unique hot spring that stands out next to it's obviously photosynthetic neighbors. This may be due to high volumes of plant matter making their way into the hot spring, but that is not confirmed. Further studies are certainly required. The mystery of Hat Stew continues!